In [1]:
!pip install --no-index --find-links="../input/commonlit-pkgs-offline-download" spacy==3.1.1

Looking in links: ../input/commonlit-pkgs-offline-download
Processing /kaggle/input/commonlit-pkgs-offline-download/spacy-3.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/srsly-2.4.1-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/typer-0.3.2-py3-none-any.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/pathy-0.6.0-py3-none-any.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/spacy_legacy-3.0.8-py2.py3-none-any.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/catalogue-2.0.4-py3-none-any.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/thinc-8.0.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installati

In [2]:
!pip install --no-index --find-links="../input/commonlit-pkgs-offline-download" fastai==2.4.1

Looking in links: ../input/commonlit-pkgs-offline-download
Processing /kaggle/input/commonlit-pkgs-offline-download/fastai-2.4.1-py3-none-any.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/torchvision-0.10.0-cp37-cp37m-manylinux1_x86_64.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/torch-1.9.0-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.1
    Uninstalling torchvision-0.8.1:
      Successfully uninstalled torchvision-0.8.1
  Attempting uninstall: fastai
    Found existing installation: fastai 2.2.7
    Uninstalling fastai-2.2.7:
      Successfully uninstalled fastai-2.2.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflic

## Prepraring for inference

### Preparing the data for inference

In [3]:
import glob
import numpy
from typing import List
from pandas.core.frame import DataFrame
from fastai.text.all import *

In [4]:
def drop_cols(df: DataFrame, cols: List) -> DataFrame:
    """drop the given list of columns from the dataframe"""
    return df.drop(cols, axis=1)

def concat_dfs(dfs: List) -> DataFrame:
    """concatenate two dataframes"""
    return pd.concat(dfs)

In [5]:
df_aug = pd.read_csv('../input/common-lit-datset-with-synonym-replacement/aug_df.csv')
df_aug = drop_cols(df_aug, ['Unnamed: 0', 'id',
                           'url_legal', 'license',
                            'standard_error'])
df_cbt_train = pd.read_csv('../input/commonlit-infusing-data-with-same-domain-data/cbt_train.csv')
df_cbt_train = drop_cols(df_cbt_train, ['Unnamed: 0'])

In [6]:
df_aug.head(1)

,excerpt,target
0,"When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThe...",-0.340259


In [7]:
df_cbt_train.head(1)

,excerpt,target
0,"With almost everything else to make them happy , they wanted one thing : they had no children .This vexed the king even more than the queen , who was very clever and learned , and who had hated dolls when she was a child .However , she , too in spite of all the books she read and all the pictures she painted , would have been glad enough to be the mother of a little prince .The king was anxious to consult the fairies , but the queen would not hear of such a thing .She did not believe in fairies : she said that they had never existed ; and that she maintained , though The History of the Roy...",NaN


Getting the excerpts to a list. This makes it easier to make predictions and later to create the submission file.

In [8]:
df_cbt_train = df_cbt_train[:10000]
excerpts = df_cbt_train.excerpt.to_list()
excerpts = excerpts
len(excerpts)

10000

## Loading the model

In [9]:
bkwd_model = load_learner('../input/common-lit-forward-backward-training-on-ulmfit/back_final_model')
fwd_model = load_learner('../input/common-lit-forward-backward-training-on-ulmfit/forward_final_model')

In [10]:
bkwd_tst_dl = bkwd_model.dls.test_dl(excerpts)
fwd_tst_dl = fwd_model.dls.test_dl(excerpts)

In [11]:
bkwd_tst_dl.show_batch()

,text
0,". xxunk a for passed he .here persia xxmaj of kingdom the of capital the was which , schiraz xxmaj at arrived travel ' days four after and , xxunk a joined brothers his with parted he after days three having , road the took , persia xxmaj into travel to designed who , brother second 's xxunk xxmaj prince xxmaj , ali xxmaj .prince came they till merchant a for passed he where and , meet to were brothers his and he which at inn the to transported were wish his formed had he as soon as and , down sat him with brought had he officer the and he , tapestry the spreading , that princess xxmaj the nearer be to eager so was he but , bisnagar xxmaj of court xxmaj and kingdom the in abode longer a made have might xxunk xxmaj .prince curiosity his satisfying"
1,". xxunk a for passed he .here persia xxmaj of kingdom the of capital the was which , schiraz xxmaj at arrived travel ' days four after and , xxunk a joined brothers his with parted he after days three having , road the took , persia xxmaj into travel to designed who , brother second 's xxunk xxmaj prince xxmaj , ali xxmaj .prince came they till merchant a for passed he where and , meet to were brothers his and he which at inn the to transported were wish his formed had he as soon as and , down sat him with brought had he officer the and he , tapestry the spreading , that princess xxmaj the nearer be to eager so was he but , bisnagar xxmaj of court xxmaj and kingdom the in abode longer a made have might xxunk xxmaj .prince curiosity his satisfying"
2,". came they till merchant a for passed he where and , meet to were brothers his and he which at inn the to transported were wish his formed had he as soon as and , down sat him with brought had he officer the and he , tapestry the spreading , that princess xxmaj the nearer be to eager so was he but , bisnagar xxmaj of court xxmaj and kingdom the in abode longer a made have might xxunk xxmaj .prince curiosity his satisfying in months some spend to and , there abode longer a make to chose he , kingdom the of religion and , customs , laws , strength the of himself inform to and , court xxmaj his and bisnagar xxmaj of king xxmaj the see to curious was he as and , agreed had they as , brothers his for there stay to obliged"
3,". came they till merchant a for passed he where and , meet to were brothers his and he which at inn the to transported were wish his formed had he as soon as and , down sat him with brought had he officer the and he , tapestry the spreading , that princess xxmaj the nearer be to eager so was he but , bisnagar xxmaj of court xxmaj and kingdom the in abode longer a made have might xxunk xxmaj .prince curiosity his satisfying in months some spend to and , there abode longer a make to chose he , kingdom the of religion and , customs , laws , strength the of himself inform to and , court xxmaj his and bisnagar xxmaj of king xxmaj the see to curious was he as and , agreed had they as , brothers his for there stay to obliged"
4,". came they till merchant a for passed he where and , meet to were brothers his and he which at inn the to transported were wish his formed had he as soon as and , down sat him with brought had he officer the and he , tapestry the spreading , that princess xxmaj the nearer be to eager so was he but , bisnagar xxmaj of court xxmaj and kingdom the in abode longer a made have might xxunk xxmaj .prince curiosity his satisfying in months some spend to and , there abode longer a make to chose he , kingdom the of religion and , customs , laws , strength the of himself inform to and , court xxmaj his and bisnagar xxmaj of king xxmaj the see to curious was he as and , agreed had they as , brothers his for there stay to obliged"
5,". him for waited ali xxmaj and xxunk xxmaj princes xxmaj the where inn the at health perfect in arrived and , indies xxmaj the to return should that xxunk first the for patiently waited , him to apple the delivered had he and , purses forty xxunk the to out counted had he 

In [12]:
fwd_tst_dl.show_batch()

,text
0,"xxbos xxmaj the shops were all of a size , and all that dealt in the same sort of goods lived in one street ; as also the xxunk - men , who kept their shops in the smaller streets .the multitude of shops , xxunk with all sorts of merchandise , as the finest xxunk from several parts of xxmaj india , some painted in the most lively colors , and representing beasts , trees , and flowers ; silks and brocades from xxmaj persia , xxmaj china , and other places , porcelain both from xxmaj japan and xxmaj china , and xxunk , surprised him so much that he knew not how to believe his own eyes ; but when he came to the xxunk and jewelers he was in a kind of xxunk to behold such prodigious quantities of wrought gold and silver , and was"
1,"xxbos xxmaj the shops were all of a size , and all that dealt in the same sort of goods lived in one street ; as also the xxunk - men , who kept their shops in the smaller streets .the multitude of shops , xxunk with all sorts of merchandise , as the finest xxunk from several parts of xxmaj india , some painted in the most lively colors , and representing beasts , trees , and flowers ; silks and brocades from xxmaj persia , xxmaj china , and other places , porcelain both from xxmaj japan and xxmaj china , and xxunk , surprised him so much that he knew not how to believe his own eyes ; but when he came to the xxunk and jewelers he was in a kind of xxunk to behold such prodigious quantities of wrought gold and silver , and was"
2,"xxbos xxmaj prince xxmaj xxunk could not view this division without admiration .it was large , and divided into several streets , all vaulted and shaded from the sun , and yet very light too .the shops were all of a size , and all that dealt in the same sort of goods lived in one street ; as also the xxunk - men , who kept their shops in the smaller streets .the multitude of shops , xxunk with all sorts of merchandise , as the finest xxunk from several parts of xxmaj india , some painted in the most lively colors , and representing beasts , trees , and flowers ; silks and brocades from xxmaj persia , xxmaj china , and other places , porcelain both from xxmaj japan and xxmaj china , and xxunk , surprised him so much that he knew not how to believe"
3,"xxbos xxmaj prince xxmaj xxunk could not view this division without admiration .it was large , and divided into several streets , all vaulted and shaded from the sun , and yet very light too .the shops were all of a size , and all that dealt in the same sort of goods lived in one street ; as also the xxunk - men , who kept their shops in the smaller streets .the multitude of shops , xxunk with all sorts of merchandise , as the finest xxunk from several parts of xxmaj india , some painted in the most lively colors , and representing beasts , trees , and flowers ; silks and brocades from xxmaj persia , xxmaj china , and other places , porcelain both from xxmaj japan and xxmaj china , and xxunk , surprised him so much that he knew not how to believe"
4,"xxbos xxmaj prince xxmaj xxunk could not view this division without admiration .it was large , and divided into several streets , all vaulted and shaded from the sun , and yet very light too .the shops were all of a size , and all that dealt in the same sort of goods lived in one street ; as also the xxunk - men , who kept their shops in the smaller streets .the multitude of shops , xxunk with all sorts of merchandise , as the finest xxunk from several parts of xxmaj india , some painted in the most lively colors , and representing beasts , trees , and flowers ; silks and brocades from xxmaj persia , xxmaj china , and other places , porcelain both from xxmaj japan and xxmaj china , and xxunk , surprised him so much that he knew not how to believe"
5,"xxbos xxmaj you shall judge yourself whether i am or no , when i have told you its property and i hope you will value it at as high a price as those i have showed it to already , who had as bad an opinion of me as you . ` ` xxmaj 

In [13]:
#bkwd_preds = bkwd_model.get_preds(dl=tst_dl)

In [14]:
#fwd_preds = fwd_model.get_preds(dl=tst_dl)

In [15]:
#p = np.array(bkwd_preds[0])
#h = np.array(fwd_preds[0])

In [16]:
#d = np.mean([p, h], axis=0)

# Predictions

In [17]:
def ensamble(excerpts, bkwd_model, fwd_model):
    bkwd_preds = np.array(bkwd_model.get_preds(dl=bkwd_tst_dl)[0])
    fwd_preds = np.array(fwd_model.get_preds(dl=fwd_tst_dl)[0])
    return np.mean([bkwd_preds, fwd_preds], axis=0)

In [18]:
predictions = [pred[0] for pred in ensamble(excerpts, bkwd_model, fwd_model)]
#predictions = [float("{0:.2f}".format(pred.item())) for pred in ensamble(bkwd_model, fwd_model, excerpts)]
#predictions = [float("{0:.2f}".format(model.predict(excerpt)[0][0])) for excerpt in excerpts]

In [19]:
predictions[:2]

[-0.7086786, -0.7086786]

# Creating the submission file

Creating a new dataframe

In [20]:
df_cbt_train = drop_cols(df_cbt_train, ['target'])

In [21]:
df_cbt_train.head(1)

,excerpt
0,"With almost everything else to make them happy , they wanted one thing : they had no children .This vexed the king even more than the queen , who was very clever and learned , and who had hated dolls when she was a child .However , she , too in spite of all the books she read and all the pictures she painted , would have been glad enough to be the mother of a little prince .The king was anxious to consult the fairies , but the queen would not hear of such a thing .She did not believe in fairies : she said that they had never existed ; and that she maintained , though The History of the Roy..."


In [22]:
df_cbt_train['target'] = predictions

In [23]:
df_cbt_train.head(1)

,excerpt,target
0,"With almost everything else to make them happy , they wanted one thing : they had no children .This vexed the king even more than the queen , who was very clever and learned , and who had hated dolls when she was a child .However , she , too in spite of all the books she read and all the pictures she painted , would have been glad enough to be the mother of a little prince .The king was anxious to consult the fairies , but the queen would not hear of such a thing .She did not believe in fairies : she said that they had never existed ; and that she maintained , though The History of the Roy...",-0.708679


In [24]:
df_pseudo_labels = concat_dfs([df_aug,df_cbt_train])

Saving the dataframe as csv .

In [25]:
df_pseudo_labels.to_csv('pseudo_labels.csv', index=False)